In [1]:
import sys  
sys.path.append('../') 

import pandas as pd
import numpy as np

from feature.feature import feature_extraction
from feature.feature import feature_normalization

In [2]:
df = pd.read_csv('../data/all.csv')

In [3]:
df.tail()

,x,y,z,class,seg,suspicious
266550,-941,174,337,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False
266551,-942,173,314,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False
266552,-916,158,310,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False
266553,-898,145,304,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False
266554,-891,132,295,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False


In [4]:
new_df = pd.read_csv('../data/processed_data/all/resample_to_50_per_act/resample_to_50.csv')

In [5]:
new_df.tail()

,x,y,z,class,seg,suspicious
251295,-928.094685,163.788475,337.651526,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False
251296,-946.717057,177.251312,326.688697,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False
251297,-933.746906,167.946609,310.380039,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False
251298,-910.344399,154.584634,309.409075,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False
251299,-896.701663,143.548819,302.866383,clean,clean/庆喜/repeat_82-duration_60.csv-1584-1628,False


In [6]:
cols = ['label', 
        'sum_x', 'sum_y', 'sum_z', \
        'mean_x', 'mean_y', 'mean_z', \
        'std_x', 'std_y', 'std_z', \
        'cv_x', 'cv_y', 'cv_z', \
        'ptp_x', 'ptp_y', 'ptp_z', \
        'p10_x', 'p10_y', 'p10_z', \
        'p25_x', 'p25_y', 'p25_z', \
        'p50_x', 'p50_y', 'p50_z', \
        'p75_x', 'p75_y', 'p75_z', \
        'p90_x',' p90_y', 'p90_z', \
        'iq_x', 'iq_y', 'iq_z', \
        'lagone_autocor_x', 'lagone_autocor_y', 'lagone_autocor_z', \
        'sk_x', 'sk_y', 'sk_z', \
        'kurt_x', 'kurt_y', 'kurt_z', \
        'power_x', 'power_y', 'power_z', \
        'logpower_x', 'logpower_y', 'logpower_z', \
        'peak_x', 'peak_y', 'peak_z', \
        'cross0_x', 'cross0_y', 'cross0_z', \
        'cor_xy', 'cor_yz', 'cor_zx']

In [7]:
%%time
training_data = pd.DataFrame(columns=cols)
segs = new_df.seg.unique()[:50]
for seg in segs:
    seg_df = new_df[new_df['seg'] == seg]
    label = seg.split('/')[0]
    x = seg_df['x'].values
    y = seg_df['y'].values
    z = seg_df['z'].values
    features = feature_extraction(x,y,z)
    len_training_data = len(training_data)
    training_data.loc[len_training_data, 'label'] = label
    training_data.loc[len_training_data, 1:] = features

Wall time: 2.57 s


In [8]:
training_data.tail()

,label,sum_x,sum_y,sum_z,mean_x,mean_y,mean_z,std_x,std_y,std_z,...,logpower_z,peak_x,peak_y,peak_z,cross0_x,cross0_y,cross0_z,cor_xy,cor_yz,cor_zx
45,walk,52693.2,11707.9,-8418.59,1053.86,234.157,-168.372,172.749,168.946,154.64,...,467.866,7,7,7,11,7,7,0.171145,-0.0149768,0.130657
46,walk,51880.4,10765.9,-5283.74,1037.61,215.318,-105.675,187.619,174.48,92.3661,...,446.86,9,9,10,14,8,12,0.0593378,0.602238,0.24646
47,walk,54449.1,10668.4,-4996.68,1088.98,213.368,-99.9336,189.873,147.285,116.893,...,449.542,7,5,8,6,6,5,0.439695,0.707922,0.104961
48,walk,53919.4,10063.4,-5678.57,1078.39,201.267,-113.571,198.093,166.008,107.179,...,444.798,7,8,10,14,8,6,0.427954,0.797621,0.296773
49,walk,53156.8,10515.3,-5930.93,1063.14,210.305,-118.619,166.418,149.976,98.8138,...,434.682,9,5,11,10,8,8,0.425451,0.844404,0.079462


In [9]:
X = np.array(training_data.drop('label',axis=1))
norm_X = feature_normalization(X)
norm_X.shape[1]

57

In [10]:
for ind in range(norm_X.shape[1]):
    training_data.iloc[:,ind+1] = norm_X[:,ind]
    
training_data.tail()

,label,sum_x,sum_y,sum_z,mean_x,mean_y,mean_z,std_x,std_y,std_z,...,logpower_z,peak_x,peak_y,peak_z,cross0_x,cross0_y,cross0_z,cor_xy,cor_yz,cor_zx
45,walk,0.144891,-0.278991,-0.206432,0.144891,-0.278991,-0.206432,-0.447679,-0.149358,0.685590,...,-0.274972,-0.199504,0.027925,-0.266865,0.895349,-0.309691,-0.443079,-0.729734,-1.839747,0.588594
46,walk,0.106624,-0.371191,0.509539,0.106624,-0.371191,0.509539,-0.330336,-0.076453,-0.531804,...,-0.793651,0.974047,0.958772,1.113471,1.868555,0.097797,0.941543,-1.233978,0.044630,0.997499
47,walk,0.227559,-0.380739,0.575099,0.227559,-0.380739,0.575099,-0.312548,-0.434709,-0.052319,...,-0.727423,-0.199504,-0.902922,0.193247,-0.726660,-0.717180,-0.996928,0.481408,0.367288,0.497858
48,walk,0.202623,-0.439960,0.419362,0.202623,-0.439960,0.419362,-0.247690,-0.188067,-0.242220,...,-0.844552,-0.199504,0.493349,1.113471,1.868555,0.097797,-0.720003,0.428455,0.641141,1.175158
49,walk,0.166717,-0.395728,0.361725,0.166717,-0.395728,0.361725,-0.497633,-0.399262,-0.405758,...,-1.094337,0.974047,-0.902922,1.573583,0.570947,0.097797,-0.166155,0.417166,0.783971,0.407821


In [11]:
norm_X.mean(axis=0)
norm_X.std(axis=0)

array([  4.13002965e-16,   6.55031585e-17,  -2.15383267e-16,
        -3.33066907e-18,  -8.54871729e-17,   7.88258347e-16,
         2.30926389e-16,  -4.18554080e-16,  -2.78665979e-16,
        -1.32567568e-16,  -1.68753900e-16,  -8.38218384e-17,
         1.99840144e-17,   5.46229728e-16,  -8.43769499e-16,
         1.27120536e-16,   1.12188037e-15,   8.43769499e-17,
        -4.24105195e-16,  -5.73985304e-16,  -7.77156117e-18,
         4.91828800e-16,   6.66133815e-17,   9.54791801e-17,
         2.16493490e-16,   4.21884749e-17,  -7.77156117e-17,
         2.56739074e-16,   3.50830476e-16,   8.49320614e-17,
         2.44249065e-17,  -2.22044605e-18,   0.00000000e+00,
         7.41073869e-16,  -6.25055563e-16,   7.54951657e-17,
        -1.33226763e-17,   3.10862447e-17,   6.99440506e-17,
         2.23500107e-13,  -3.66272568e-13,  -1.96566652e-13,
         8.38218384e-16,   1.12132525e-16,   0.00000000e+00,
         5.26134691e-15,   6.08402217e-16,  -1.07469589e-15,
         5.32907052e-17,

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.])